## Introduction to Natural Language Processing
[**CC-BY-NC-SA**](https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en)<br/>
Prof. Dr. Annemarie Friedrich<br/>
Faculty of Applied Computer Science, University of Augsburg<br/>
Date: **SS 2025**

# 11. Sequence Labeling (Homework)

__Recommendation:__ Use a GPU for notebook, e.g., in Google Colab Runtime --> Change Runtime --> GPU --> T4.

**Learning Goals**

* Explain part-of-speech tagging
* Explain named entity recognition
* Implement masking in PyTorch
* Train and evaluate a sequence labeling model

❗ Upon completion, upload your code (this notebook) to your GitLab repository.

In [2]:
# Installations
!pip install -U datasets==3.6.0
!pip install transformers
!pip install seqeval
!pip install evaluate

# Imports
import numpy as np
import random
import os

import torch
import transformers
import evaluate
from torch import optim
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

# Define the device we'll use for tensor computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Computing on:", device)

# Should we still have some source for non-determinism in our code, this will complain:
torch.use_deterministic_algorithms(True, warn_only=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

## Named Entity Recognition

In this homework, we will work with the CoNLL 2003 Named Entity dataset. The dataset was developed for a shared task described in [Introduction to the CoNLL-2003 Shared Task: Language-Independent Named Entity Recognition (Tjong Kim Sang & De Meulder, CoNLL 2003)](https://aclanthology.org/W03-0419/).

❓ Check out the [Dataset card of the CoNLL 2003 dataset](https://huggingface.co/datasets/conll2003) on HuggingFace models.

❓ If you are still unsure how the BIO scheme works, now is the time to do a brief web search and figure it out! Advanced: The BILOU variant is currently achieving state-of-the-art results. Read about it.

In [3]:
# Load the dataset

# A July 9th Update to the datasets package broke this, so make sure to use datasets==3.6.0
train_data = load_dataset("conll2003", split="train")
val_data =  load_dataset("conll2003", split="validation")
test_data =  load_dataset("conll2003", split="test")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

conll2003.py: 0.00B [00:00, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

❓ Print out the number of instances in each datasplit.

In [4]:
print(f"Instances in split 'train': {len(train_data)}")
print(f"Instances in split 'validation': {len(val_data)}")
print(f"Instances in split 'test': {len(test_data)}")

Instances in split 'train': 14041
Instances in split 'validation': 3250
Instances in split 'test': 3453


The NER tags in the CoNLL 2003 NER dataset are (with their class indices):

```
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
```

❓ Create a list `ner_tags` of the labels in the order indicated by their values in the dictionary above. Create two dictionaries `tag2idx` and `idx2tag` that map from tag to class index and from class index to tag.

In [5]:
ner_tags = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
# Set up dicts to go back and forth from label to idx
label2idx = {l:i for i, l in enumerate(ner_tags)}
print(label2idx)
idx2label = {i:l for i, l in enumerate(ner_tags)}
print(idx2label)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


❓ LOOK AT THE DATA. Print out a few examples and make sure you understand the data structure.

Example Output (training set instance with index 189):
```
Iraqi           B-MISC
President       O    
Saddam          B-PER
Hussein         I-PER
has             O    
told            O    
visiting        O    
Russian         B-MISC
ultra-nationalist O    
Vladimir        B-PER
Zhirinovsky     I-PER
that            O    
Baghdad         B-LOC
wanted          O    
to              O    
maintain        O    
"               O    
friendship      O    
and             O    
cooperation     O    
"               O    
with            O    
Moscow          B-LOC
,               O    
official        O    
Iraqi           B-MISC
newspapers      O    
said            O    
on              O    
Thursday        O    
.               O    
```

In [6]:
print(train_data[11])
print(val_data[21])
print(test_data[3])

{'id': '11', 'tokens': ['.'], 'pos_tags': [7], 'chunk_tags': [0], 'ner_tags': [0]}
{'id': '21', 'tokens': ['Leicestershire', '22', 'points', ',', 'Somerset', '4', '.'], 'pos_tags': [22, 11, 24, 6, 22, 11, 7], 'chunk_tags': [11, 12, 12, 0, 11, 12, 0], 'ner_tags': [3, 0, 0, 0, 3, 0, 0]}
{'id': '3', 'tokens': ['Japan', 'began', 'the', 'defence', 'of', 'their', 'Asian', 'Cup', 'title', 'with', 'a', 'lucky', '2-1', 'win', 'against', 'Syria', 'in', 'a', 'Group', 'C', 'championship', 'match', 'on', 'Friday', '.'], 'pos_tags': [22, 38, 12, 21, 15, 29, 16, 22, 21, 15, 12, 16, 11, 41, 15, 22, 15, 12, 22, 22, 21, 21, 15, 22, 7], 'chunk_tags': [11, 21, 11, 12, 13, 11, 12, 12, 12, 13, 11, 12, 12, 21, 13, 11, 13, 11, 12, 12, 12, 12, 13, 11, 0], 'ner_tags': [5, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]}



### Tokenization

For tokenization, we use the [AutoTokenizer](https://huggingface.co/docs/transformers/model_doc/auto) class of the HuggingFace transformers library. It conveniently instantiates an object of the correct class depending on the model configured in the path.
The parameter setting `add_special_tokens=True` means that the `[CLS]` and the `[SEP]` tokens are added to the input. Even if we are not performing sentence classification or sentence pair classification, we have to add these tokens because BERT saw them during pre-training, and now expects to see them, too.

To save some computational resources, we will work with [TinyBERT](https://arxiv.org/abs/1909.10351) using the [model](https://huggingface.co/prajjwal1/bert-tiny) provided by [Bhargava et al., 2021](https://aclanthology.org/2021.insights-1.18/). TinyBERT has been trained using distillation to mimick the behavior of BERT, but it is a much smaller and more efficient model.

If `tokenizer.is_fast` is `True`, we are using a [Fast tokenizer](https://huggingface.co/learn/nlp-course/chapter6/3) that provides some special features that are quite handy when working with pre-trained models.

* As input, a fast tokenizer accepts either a string (a single input instance), a list of strings (input texts), or a list of a list of tokens. The latter is particularly helpful if our data is already pretokenized. In the CoNLL 2003 dataset, this is the case. (Note that we will use a gold standard tokenization for the "real" tokens. In a real-world setting with an automatic tokenizer, performance might be less. However, as long as we note this in our experimental section as a caveat, doing this is accepted nowadays because tokenizers work pretty well for many domains and genres.)

* If given more than one instance, the fast tokenizer automatically process the inputs in a batch, being MUCH fast than when processing individual instances. (Hint (optional exercise): Modify the code below such that it only processes one sentence at a time and see how much slower it gets.)

* The tokenizer accepts a wide range of useful parameter settings. The `return_tensors='pt'` options means that we want to get back PyTorch tensors. We add special tokens and we also want to get attention masks. Finally, we tell the tokenizer that the input is already split into words such that it does not attempt to create a tokenization that would contradict the given tokenization, which would be problematic if we want to map the outputs to the gold standard labels. Can you figure out what the `truncation` and the `padding` options do?

In [7]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny") # Load the tokenizer that comes with the TinyBERT model.
print("Is fast encoder (should be True):", tokenizer.is_fast)

input_examples = [["I", "visited", "the", "Augsburger", "Puppenkiste", "."], \
                  ["The", "Rathaus", "is", "much", "more", "interesting", "in", "my", "opinion", "."]]
encodings = tokenizer(input_examples, return_tensors='pt', add_special_tokens=True, \
                      return_attention_mask=True, is_split_into_words=True, \
                      truncation=True, padding=True, max_length=32)

print(type(encodings))
print(encodings)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Is fast encoder (should be True): True
<class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': tensor([[  101,  1045,  4716,  1996, 24362,  2121, 26781, 11837, 14270,  2618,
          1012,   102,     0],
        [  101,  1996,  9350, 13821,  2003,  2172,  2062,  5875,  1999,  2026,
          5448,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Inspect the data structures returned by the tokenizer. They are of type [`BatchEncoding`](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/tokenizer#transformers.BatchEncoding).
The maximum length of the sequence has been determined based on the maximum input length of the training set / maximum model input size / the value of max_length that we defined.

The BatchEncoding object can be viewed as a dictionary where each entry contains a tensor that contains the data for one input sequence per row, i.e., the `input_ids`, `attention_mask` etc. of each instance are split across these tensors. However, the same row in these tensors always corresponds to the same instance.

You can print them out as follows.

In [8]:
num = 0 # Change to 1 and check the outputs.
print(encodings["input_ids"][num])
print(encodings["attention_mask"][num])
print(encodings.tokens(num)) # This prints the word piece tokens (input_ids --> convert_to_token_ids)

tensor([  101,  1045,  4716,  1996, 24362,  2121, 26781, 11837, 14270,  2618,
         1012,   102,     0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
['[CLS]', 'i', 'visited', 'the', 'augsburg', '##er', 'pup', '##pen', '##kis', '##te', '.', '[SEP]', '[PAD]']


Now, let's assume that our gold standard labels are as follows:

```
I           O
visited     O
the         O
Augsburger  B-ORG
Puppenkiste I-ORG
.           O
```

At this point, we have a problem: The original labels above relate to the "real" tokens, but the word-piece tokens have split up some of them into several tokens. Recall that we have only labels for "real" tokens. There are two possible strategies to address this:

1. Use the first subword token of a "real" token to represent the embedding, only compare the predicted label for this token to the gold standard token and back-propagate the loss accordingly only from these tokens. This can be achieved by setting the labels of all non-used subword tokens to `-100`, a magic label index ignored by PyTorch loss functions (i.e., no loss is backpropagated from these outputs).

2. Duplicate the label of the "real" token for all its subword tokens.

Today, we will implement version 1. Luckily, the `BatchEncoding` object has a method called `word_ids` which returns a list showing to which "real" token a subword token corresponds. Each entry in this list corresponds to the subword token at the same position. The numbers in this list indicate the position of the corresponding "real" token in the original sequence.

❓ Make sure you understand the output of the code cell below.

In [9]:
num = 0 # change to 1 to inspect data
print([(i, w) for i, w in enumerate(input_examples[num])])
print(encodings.tokens(num))
word_ids = encodings.word_ids(num)
print(word_ids)
print([(orig_word_id, subword_token) for orig_word_id, subword_token in zip(word_ids, encodings.tokens(num))])

[(0, 'I'), (1, 'visited'), (2, 'the'), (3, 'Augsburger'), (4, 'Puppenkiste'), (5, '.')]
['[CLS]', 'i', 'visited', 'the', 'augsburg', '##er', 'pup', '##pen', '##kis', '##te', '.', '[SEP]', '[PAD]']
[None, 0, 1, 2, 3, 3, 4, 4, 4, 4, 5, None, None]
[(None, '[CLS]'), (0, 'i'), (1, 'visited'), (2, 'the'), (3, 'augsburg'), (3, '##er'), (4, 'pup'), (4, '##pen'), (4, '##kis'), (4, '##te'), (5, '.'), (None, '[SEP]'), (None, '[PAD]')]


### Perform Tokenization and Creating Label Tensors

❓ Write a function that tokenizes the text of each input example using the tokenizer in the configuration as above. Use a `max_length` of 64. The original datasets provide the label information (NER tags) in the field `"ner_tags"`. Indices in this list correspond to the pre-tokenized "real" tokens. Create list of labels for each instance that contains the integer values of the class indices for each subwork token. Follow the idea that all subword tokens that do not correspond to the first subword token of a "real" token should get a label of `-100`. Tokenize and create label lists for each of the data splits.

In [10]:
def create_token_label_tensors(data_set):

  # We'll be returning a list with a list_of_tags for every instance in data_set
  # Each list_of_tags will have an int tag for "main tokens"; -100 for subtokens and BERT tokens like 'CLS'
  tags_all_instances_padded = []

  # Load the tokenizer, create the encodings object
  tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
  print(f"Is fast encoder (should be True): {tokenizer.is_fast}.")
  encodings = tokenizer([i['tokens'] for i in data_set], return_tensors='pt', add_special_tokens=True, \
                      return_attention_mask=True, is_split_into_words=True, \
                      truncation=True, padding=True, max_length=64)

  # encodings.tokens(i) contains the tokenizer tokens for a whole instance
  # encodings.word_ids(i) contains the "parent token indexes"  for a whole instance: what words in the original sequence each subword maps to
  for i in range(len(data_set)):
    tags_of_instance = []
    for orig_word_id, subword_token in zip(encodings.word_ids(i), encodings.tokens(i)):
      # a tokenizer token that maps to no original token gets -100
      if orig_word_id is None:
        tags_of_instance.append(-100)
      # a tokenizer token that matches the beginning of its original token gets the original's label
      elif data_set[i]['tokens'][orig_word_id].lower().startswith(subword_token):
        tags_of_instance.append(data_set[i]['ner_tags'][orig_word_id])
      # a tokenizer token that does not match the beginning of its original token is assumed to be a subword --> gets -100
      else:
        tags_of_instance.append(-100)

    tags_all_instances_padded.append(tags_of_instance)

  # for debugging, feel free to comment in/out
  #num = 9
  #print(encodings.tokens(num))
  #print(encodings.word_ids(num))
  #print([(orig_word_id, subword_token) for orig_word_id, subword_token in zip(encodings.word_ids(num), encodings.tokens(num))])
  #print(tags_all_instances_padded[num])

  return encodings.convert_to_tensors(), tags_all_instances_padded

train_enc, train_labels = create_token_label_tensors(train_data)
val_enc, val_labels = create_token_label_tensors(val_data)
test_enc, test_labels = create_token_label_tensors(test_data)


Is fast encoder (should be True): True.
Is fast encoder (should be True): True.
Is fast encoder (should be True): True.


In [11]:
print(type(train_enc))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


### Custom PyTorch Dataset for NER

❓ Write a custom PyTorch dataset, e.g., called `NERDataset` that inherits from `Dataset` and implements the `__init__`, `__len__`, and `__getitem__` methods. (Hint: Check out the notebooks from the last session if you are unsure how to do this.)

In [12]:
# Preparing a custom dataset for BERT
class NERDataset(Dataset):
    def __init__(self, encodings, labels):
      self.input_ids = encodings["input_ids"]
      self.input_ids = self.input_ids.to(device=device)

      self.token_type_ids = encodings["token_type_ids"]
      self.token_type_ids = self.token_type_ids.to(device=device)

      self.attention_mask = encodings["attention_mask"]
      self.attention_mask = self.attention_mask.to(device=device)

      self.labels = torch.tensor(labels, dtype=torch.long, device=device)

    def __getitem__(self, idx):
      label = torch.clone(self.labels[idx])
      item = {"input_ids" : torch.clone(self.input_ids[idx]),
              "token_type_ids" : torch.clone(self.token_type_ids[idx]),
              "attention_mask" : torch.clone(self.attention_mask[idx])}
      return item, label

    def __len__(self):
        return len(self.labels)

train_dataset = NERDataset(train_enc, train_labels)
val_dataset = NERDataset(val_enc, val_labels)
test_dataset = NERDataset(test_enc, test_labels)

### Adapting BERT for NER

❓ Next, complete the code for the `BertNerModel` below. The `forward` method should use one linear layer that outputs one logit per valid label (referring to the `ner_tags`, -100 is not a valid label). The input to this linear layer is simply the `last_hidden_state` of the BERT model.

In [13]:
hidden_size = 128
num_tags = len(ner_tags)

class BertNerModel(torch.nn.Module):

  def __init__(self):
    super(BertNerModel, self).__init__()
    self.bert = AutoModel.from_pretrained("prajjwal1/bert-tiny")
    self.linear = torch.nn.Linear(hidden_size, num_tags)

  def forward(self, inputs):
    outputs = self.bert(**inputs, output_hidden_states=True)
    last_hidden_state = outputs.hidden_states[-1]
    logits = self.linear(last_hidden_state)

    return logits


### Evaluation

Now, before we actually start the learning, we should define how we measure performance.

❓ Could we simply use token-wise accuracy? Why is this not meaningful?

❓ Work through [Evaluate sequence models in python](https://www.depends-on-the-definition.com/evaluate-sequence-models/) by Torbias Sterbak.
We can load the `seqeval` metric in PyTorch as follows.

```
import evaluate
metric = evaluate.load("seqeval")
```

The `metric` object can be called as follows:

`metric.compute(predictions=predictions, references=true_labels)`

Note that you should ignore any tokens that have a label of `-100`, i.e., from the model's output, you first need to create the `predictions` and `true_labels` lists. For example, assume the model output and the list of true labels (that you created above) look like this:

```
gold_labels =      [-100, 0, 0, 2, 3, -100, 3, 0, 4, -100, 0, 7, -100]
predicted_labels = [9,    0, 0, 1, 3,    3, 3, 0, 5,    5, 0, 7,    0]
```

You need to filter this list of labels such that all the `-100` values are removed:
```
gold_labels =      [0, 0, 2, 3, 3, 0, 4, 0, 7]
predicted_labels = [0, 0, 1, 3, 3, 0, 5, 0, 7]
```

❓ Implement a function (or two if you prefer) that (given a model):

1. Collect the gold standard labels and the predicted labels for all instances of a given dataset (hint: assume that the input is a DataLoader).

2. Filter the lists for valid tokens and labels as described above.

3. Load the sequeval metric and use `metric.compute` to compute class-wise and overall evaluation scores.

IMPORTANT HINT: Read through the next exercise first (implementation of the training loop) or adapt your functions later to make it fit to the model outputs.

In [14]:
def flat_list(list):
  flat = []
  for batch in list:
    for item in batch:
      flat.append(item)
  return flat

In [18]:
import evaluate
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

# Compile two parallel lists of ONLY entries that are NOT -100 on the gold list
def filter_lists(gold, predicted):
  # Check that the lists have the same length ("number of rows")
  if(len(gold) != len(predicted)):
    print(f"Error! The lists have different lengths!")
    return None, None

  gold_f = []
  predicted_f = []

  for g, p in zip(gold, predicted):
    g_f = []
    p_f = []
    # Check that the lists have the same length ("number of cols per row")
    if(len(g) != len(p)):
      print(f"Error! The lists have different lengths!")
      return None, None

    for i in range(len(g)):
      if(g[i] != -100):
        g_f.append(idx2label[g[i].item()])
        # reducing the tensor with 9 scores to a tensor with the argmax of the former
        p_f.append(idx2label[torch.argmax(p[i]).item()])
    gold_f.append(g_f)
    predicted_f.append(p_f)
    #print("gold")
    #print(gold_f)
    #print("predicted")
    #print(predicted_f)
  return gold_f, predicted_f

def evaluate_model(model, data_loader):
  with torch.no_grad(): # This tells the model that we're not training, i.e it will not remember gradients for this block
    goldtags = []
    predictions = []

    model.eval()

    for X, y in iter(data_loader):
      g, p = filter_lists(y, model(X))
      goldtags.append(g)
      predictions.append(p)

    gold_flat = flat_list(goldtags)
    pred_flat = flat_list(predictions)

    metric = evaluate.load("seqeval")
    #metric.compute(predictions=predictions, references=goldtags)
    print(classification_report(gold_flat, pred_flat))




### Training

At the beginning of your training cell, include the following code for setting the random seeds. (It is important to execute any time you start the training to ensure the random seeds are reset at this point in time.)

❓ Implement the training loop (adapt if from last week's notebook). Use AdamW as optimizer (`optimizer = AdamW(model.parameters(), lr=learning_rate, betas=betas, eps=epsilon)`).
Suggested parameters:

```
num_epochs = 16
batch_size = 64
learning_rate = 3e-5
betas=(0.9,0.999)
epsilon=1e-08
```

Make sure to shuffle the training data in your training DataLoader.
The training takes about X minutes on a GPU T4 in Google Colab with my implementation, and the results for the test set are as follows (yours might differ a bit due to some randomness in the loss function works).

```
LOC
	      precision 0.8064516129032258
	         recall 0.8750754375377188
	             f1 0.8393632416787264
	         number 1657
MISC
	      precision 0.6221928665785997
	         recall 0.6709401709401709
	             f1 0.6456477039067854
	         number 702
ORG
	      precision 0.6782922429344558
	         recall 0.6807483403741702
	             f1 0.6795180722891565
	         number 1657
PER
	      precision 0.8291062801932367
	         recall 0.872299872935197
	             f1 0.8501547987616099
	         number 1574

	overall_precision 0.7528089887640449
	 overall_recall 0.7910554561717352
	     overall_f1 0.7714584787159804
	overall_accuracy 0.9562975321214222
```

As loss function, use `torch.nn.CrossEntropyLoss()`, which INCLUDES the softmax already. When you determine the predictions from the output in your evaluation functions above, you do this simply by finding the argmax over the logit values.

Here is a snippet of PyTorch code that you may find useful:

```
    y_pred = model(X)  # Have our model with current weights make a prediction
    # outputs should be of shape [batch, sequence, logits] (where sequence values indicate the token indices within one sequence)
    y_pred = torch.permute(y_pred, (0, 2, 1))  # swap the sequence and the logit dimensions
    loss = loss_fn(y_pred, y)  # ... sucht that the loss function can take care of the rest for us!
```


In [19]:
# Always fun with the random seeds ...
# We need to set them such that our results will be replicable.
# (Hint: for an experiment later, you can change the random seed here and check what happens.
# But for now, let's keep the answer to all questions of the universe, 42.)
seed=42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
if torch.cuda.is_available():
  # This is needed on Colab as we are working in a distributed environment
  # If you are working in a different GPU environment, you can probably omit this line if it results in errors.
  os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"


#####################################
# Instantiate the model             #
#####################################

model = BertNerModel()
model = model.to(device)

#####################################
# Training / Fine-tuning the model  #
#####################################

num_epochs = 16
batch_size = 64
learning_rate = 3e-5
betas=(0.9,0.999)
epsilon=1e-08

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, betas=betas, eps=epsilon)
loss_fn = torch.nn.CrossEntropyLoss()

data_loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
data_loader_val = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
data_loader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training
for n in range(num_epochs):
  model.train()
  it = iter(data_loader_train)  # Create the iterator from the training dataset
  epoch_loss, steps = 0, 0      # To keep track of the current epoch's loss

  for  X, y in it:              # Obtain a tensor X = batch of X-values, y accordingly

    y_pred = model(X)           # Have our model with current weights make a prediction
                                # outputs should be of shape [batch, sequence, logits] (where sequence values indicate the token indices within one sequence)
    y_pred = torch.permute(y_pred, (0, 2, 1))  # swap the sequence and the logit dimensions
    loss = loss_fn(y_pred, y)  # ... such that the loss function can take care of the rest for us!

    optimizer.zero_grad()       # Reset the optimizer (otherwise it accumulates results - would be wrong here)
    loss.backward()             # Compute the gradients (partial derivatives)
    optimizer.step()            # Update the network's weights
    epoch_loss += loss          # For tracking the epoch's loss
    steps += 1

  print("\nEpoch:", n+1, "    Loss: {:0.4f}".format(epoch_loss/steps))
  # evaluate model at end of epoch
  print("Training report:")
  evaluate_model(model, data_loader_train)
  print("Validation report:")
  evaluate_model(model, data_loader_val)


Epoch: 1     Loss: 0.7243
Training report:
              precision    recall  f1-score   support

         LOC       0.67      0.46      0.55      7135
        MISC       0.45      0.03      0.06      3437
         ORG       0.42      0.18      0.25      6316
         PER       0.24      0.29      0.26      6585

   micro avg       0.40      0.28      0.33     23473
   macro avg       0.44      0.24      0.28     23473
weighted avg       0.45      0.28      0.32     23473

Validation report:
              precision    recall  f1-score   support

         LOC       0.69      0.45      0.55      1838
        MISC       0.46      0.04      0.07       919
         ORG       0.46      0.20      0.28      1343
         PER       0.24      0.27      0.25      1790

   micro avg       0.42      0.27      0.33      5890
   macro avg       0.46      0.24      0.29      5890
weighted avg       0.46      0.27      0.32      5890


Epoch: 2     Loss: 0.3750
Training report:
              precision

In [ ]:
# Evaluate on the test set
# Your code here

❗ Upon completion, upload your code (this notebook) to your GitLab repository.